# evaluate WER of whisper with PEFT LoRA

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phineas-pta/fine-tune-whisper-vi/blob/main/eval/evaluate-whisper-lora.ipynb)

*kaggle TPU crash when running inference* ~~can be used on kaggle TPU, but do not enable `XLA_USE_BF16` because of AMP (Automatic Mixed Precision)~~

try `transformers.pipeline` but error with `torch.autocast`

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
# !huggingface-cli login --token=███

In [ ]:
# workaround for a bug in `datasets` package
%pip uninstall -y cudf dask-cuda dask-cudf
%pip install -q cudf-cu12 --extra-index-url=https://pypi.nvidia.com
%pip install -qU 'datasets[audio]' accelerate transformers jiwer bitsandbytes peft
# install then `import evaluate` throw error on kaggle

In [ ]:
from tqdm import tqdm
import torch
# import torch_xla.core.xla_model as xm  # on kaggle TPU
from peft import PeftModel, PeftConfig
from transformers import AutomaticSpeechRecognitionPipeline, WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer
import datasets as hugDS
import jiwer

JIWER_TRANS = jiwer.Compose([  # DO NOT use `jiwer.RemoveEmptyStrings` it can cause rows count mismatch
	jiwer.ToLowerCase(),
	jiwer.RemoveKaldiNonWords(),
	jiwer.RemoveMultipleSpaces(),
	jiwer.Strip(),
	jiwer.RemovePunctuation(),
	jiwer.ReduceToListOfListOfWords(),
])

In [ ]:
SAMPLING_RATE = 16_000
def load_my_data(**kwargs):
	return hugDS.load_dataset(**kwargs, split="test", trust_remote_code=True, streaming=True).cast_column("audio", hugDS.Audio(sampling_rate=SAMPLING_RATE))

MY_DATA = hugDS.IterableDatasetDict()
MY_DATA["commonvoice"] = load_my_data(path="mozilla-foundation/common_voice_16_1", name="vi",  ).select_columns(["audio", "sentence"])
MY_DATA["fleurs"]      = load_my_data(path="google/fleurs",                        name="vi_vn").select_columns(["audio", "transcription"]).rename_column("transcription", "sentence")
MY_DATA["vivos"]       = load_my_data(path="vivos"                                             ).select_columns(["audio", "sentence"])
MY_DATA["bud500"]      = load_my_data(path="linhtran92/viet_bud500"                            ).rename_column("transcription", "sentence")
MY_DATA["lsvsc"]       = load_my_data(path="doof-ferb/LSVSC"                                   ).select_columns(["audio", "transcription"]).rename_column("transcription", "sentence")

ROWS_COUNT = {
	"commonvoice": 1326,
	"fleurs":       857,
	"vivos":        760,
	"bud500":      7500,
	"lsvsc":       5683,
}

In [ ]:
PEFT_MODEL_ID = "doof-ferb/whisper-large-peft-lora-vi"  # @param ["doof-ferb/whisper-large-peft-lora-vi", "daila/whisper-large-v3_LoRA_Common-Vi_WER", "daila/whisper-large-v3_LoRA_vi", "vikas85/whisper-vlsp-peft", "vikas85/whisper-vlsp", "vikas85/whisper-fosd-peft", "vikas85/whisper-fleurs-peft-vi-2", "DuyTa/vi-whisper-medium-Lora", "vikas85/whisper-cv-fleur-v6", "vikas85/fleurs-vn-peft-v2", "Yuhthe/openai-whisper-small-vivos-LORA-colab"]
BASE_MODEL_ID = PeftConfig.from_pretrained(PEFT_MODEL_ID).base_model_name_or_path
print("adapter to", BASE_MODEL_ID)

# declare task & language in extractor & tokenizer have no effect in inference
FEATURE_EXTRACTOR = WhisperFeatureExtractor.from_pretrained(BASE_MODEL_ID)
TOKENIZER = WhisperTokenizer.from_pretrained(BASE_MODEL_ID)

MODEL = PeftModel.from_pretrained(
	WhisperForConditionalGeneration.from_pretrained(
		BASE_MODEL_ID,
		device_map="auto",  # {"": "cuda:0"}  # {"": xm.xla_device()}  # on kaggle TPU
		torch_dtype=torch.float16,  # bfloat16 on TPU or newer GPU
		# quantization_config=BitsAndBytesConfig(load_in_8bit=True)  # do not enable because it make inference super slow
	),  # do not use `.to("cuda")` because throw weird error about torch dtype mismatch
	PEFT_MODEL_ID
).merge_and_unload(progressbar=True)  # reduce latency with LoRA

PIPE = AutomaticSpeechRecognitionPipeline(model=MODEL, tokenizer=TOKENIZER, feature_extractor=FEATURE_EXTRACTOR)
PIPE_KWARGS = {"language": "vi", "task": "transcribe", "do_sample": True, "num_beams": 5}

BATCH_SIZE =  16 # @param {type: "integer"}
# @markdown for colab free T4 @ `float16`: 16 for large model, 32 medium, 48 small

In [ ]:
# workaround because KeyDataset(MY_DATA[split], "audio") raise error with streaming datasets
def data(batch):
	for row in batch:
		yield row["audio"]


@torch.autocast(device_type="cuda")  # required by PEFT
# @torch.autocast(device_type="xla", dtype=torch.bfloat16)  # on kaggle TPU
@torch.inference_mode()
def predict(split):
	batch = MY_DATA[split]
	y_pred = [out["text"] for out in tqdm(PIPE(data(batch), generate_kwargs=PIPE_KWARGS, batch_size=BATCH_SIZE), total=ROWS_COUNT[split], unit="samples", desc=f"{split=}")]
	torch.cuda.empty_cache()  # forced clean
	y_true = [row["sentence"] for row in batch]
	return hugDS.Dataset.from_dict({"true": y_true, "pred": y_pred})


MY_DATA_BIS = hugDS.DatasetDict()  # do not use MY_DATA.map() because later need non-iterable with jiwer
for split in MY_DATA.keys():
	MY_DATA_BIS[split] = predict(split)

In [ ]:
for split in MY_DATA_BIS.keys():
	wer = 100 * jiwer.wer(
		reference=MY_DATA_BIS[split]["sentence"], hypothesis=MY_DATA_BIS[split]["pred"],
		reference_transform=JIWER_TRANS,          hypothesis_transform=JIWER_TRANS,
	)
	print(f"WER on {split} = {wer:.1f}%")